In [122]:
import pandas as pd
pd.set_option("display.float_format", lambda x: "%.2f" % x)

In [123]:
import tkinter as tk
from tkinter import filedialog

In [124]:
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()

In [125]:
df = pd.read_excel(file_path, sheet_name= "出貨明細")

In [126]:
sort_df = df[df["預交日期"] >= "2020-01-01"]

# 全系列商品+全客戶的購物籃分析

In [145]:
order_numbers = sort_df["銷售單號"].unique().tolist()

In [146]:
len(sort_df["銷售單號"].unique())

2037

In [147]:
104 / 2037

0.051055473735886106

In [149]:
cart_ordered = []
for i in order_numbers:
    order_list = sort_df[sort_df["銷售單號"] == i]["品名"].values
    cart_ordered.append(order_list)

In [165]:
# 整理出所有每筆訂單的購物內容
cart_ordered

[array(['MIP1608P1R0MBP', 'MIP1608P2R2MBP', 'MIP1608PR47MBP',
        'WIP201610P-2R2ML', 'WIP201610P-R33ML', 'WIP252012P-2R2ML',
        'WIPC252012P-1R0ML', 'WIP252012P-R47ML'], dtype=object),
 array(['MLVS1206AM14701F', 'MLVS1206AM35201F', 'MLVS0805AM08361F',
        'MLVS0805AM30201F', 'MLVS0805AM30201F', 'MLVS1206AM30501F',
        'MLVS1206AM50251F'], dtype=object),
 array(['EGA10603V05B0', 'EGA10603V05B0', 'EGA10603V12B0', 'EGA10603V12B0',
        'EGA10603V12B0'], dtype=object),
 array(['MLVS0402AM10101', 'MLVS0603AM04301F', 'MLVS0603AM04301F',
        'MLVS0603AM04301F', 'MLVS1206AM30501F', 'MLVS0805AM06421F',
        'MLVS0805AM20221F', 'MLVS0805AM25251F', 'MLVS0805AM30201F',
        'MLVS0603AM30350F'], dtype=object),
 array(['WIP201612S-R47ML'], dtype=object),
 array(['WIPEA252012S-1R0ML'], dtype=object),
 array(['EGA10603V12B0'], dtype=object),
 array(['MCB1005S300FBP', 'MIP20121R0MBE', 'MHC1608S600QBP',
        'MCB1608S152HBP'], dtype=object),
 array(['MIP20121R0MBE'], d

In [152]:
from mlxtend.preprocessing import TransactionEncoder

In [166]:
te = TransactionEncoder()
te.ary = te.fit(cart_ordered).transform(cart_ordered)

In [167]:
te.ary

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [168]:
te.columns_

['000005700008 SurgX0603ESDA2',
 '000006001072 MLVS1206AM30501F',
 '056ZEN075（RS1010）-SMT',
 '056ZEN130 New(RS4146)-SMT',
 '056ZEN230（RS1013）-SMT',
 '059ZEN130（RS1362）-SMT',
 '065ZEN130（RS1007）-SMT',
 '065ZEN230（RS1006）-SMT',
 '132ZEN130（RS1005）-SMT',
 '132ZEN230（RS1009）-SMT',
 'ACA-3216-D1-MF-S',
 'ACA-5036-A2-CC-S-00-E1',
 'ACA-5036-A7-CC-S',
 'ACD-5036-A3-CC-S-00-E1',
 'ACE-3505-P1-PF-S',
 'ACE-3505-P1-PF-STestBoard',
 'ACH-5036-A1-CC-S-00-E1',
 'ACM0-3216-A1-CC-S',
 'ACM0-3216-A1-CC-S-00',
 'ACM0-3216-A1-CC-S-00-E1',
 'ACM3-5036-A1-CC-S',
 'ACM3-5036-A1-CC-S-00-E1',
 'ACM3-5036-A1-CC-S-chip',
 'ACM4-5036-A1-CC-S-00',
 'AIP0530GX-R33MPB',
 'AIP06030GX Series SPEC',
 'AIP0603GX-100MPB',
 'AIP0603GX-1R0MPB',
 'AIP0603GX-1R8MPB',
 'AIP0603GX-220MPB',
 'AIP0603GX-2R2MPB',
 'AIP0603GX-3R3MPB',
 'AIP0603GX-4R7MPB',
 'AIP0603GX-6R8MPB',
 'AIP1040EX-1R5MPA',
 'AIP1040EX-3R3MPA',
 'B3G02G-S3-01-A',
 'C03粉末',
 'CES10201V24B0',
 'CGA10201V05B0',
 'CI060303-3N9C*30kpcs(HOTAYI-TI',
 'DAM-P11-B-R

In [169]:
test_frame = pd.DataFrame(te.ary, columns=te.columns_)

# 購物籃分析術語解釋
1. 支持度（Support）：這個商品組合在所有購買清單中，出現的機率。
2. 信賴度（Confidence）：購買A商品後，購買B商品的機率，這也是本篇文章最後要得知的結果。
3. 提昇度（Lift）：判斷A商品與B商品的相關程度，若小於1表示負相關，等於1表示完全無相關（完全沒有交集），大於1表示正相關。

In [183]:
from mlxtend.frequent_patterns import apriori
frequent_items = apriori(test_frame, min_support=0.02, use_colnames=True)
frequent_items['length'] = frequent_items['itemsets'].apply(lambda x: len(x))
# frequent_items = frequent_items[frequent_items["length"] == 2]

In [184]:
frequent_items

,support,itemsets,length
0,0.02,(EGA10402V05AH),1
1,0.04,(EGA10603V05B0),1
2,0.02,(EGA10603V12A0),1
3,0.03,(EGA10603V12B0),1
4,0.03,(MHC1005G121MBP),1
5,0.02,(MHC1608P260Z06BP6A0),1
6,0.03,(MHC3216S601PBE),1
7,0.02,(MIP25202R2MBE),1
8,0.02,(MIP25204R7MBE),1
9,0.02,(MLVS0603L04),1


In [187]:
from mlxtend.frequent_patterns import association_rules
result = association_rules(frequent_items, metric="confidence", min_threshold=0.8)

In [188]:
result.sort_values("confidence", ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(WAG-F-LAG0-00-042-I),(WAG-F-LTE10-00-052-I),0.02,0.02,0.02,1.00,47.37,0.02,inf
1,(WAG-F-LTE10-00-052-I),(WAG-F-LAG0-00-042-I),0.02,0.02,0.02,1.00,47.37,0.02,inf
2,(WAG-F-LAG0-00-042-I),(WAG-F-LTE10-00-053-I),0.02,0.02,0.02,1.00,47.37,0.02,inf
3,(WAG-F-LTE10-00-053-I),(WAG-F-LAG0-00-042-I),0.02,0.02,0.02,1.00,47.37,0.02,inf
4,(WAG-F-LTE10-00-052-I),(WAG-F-LTE10-00-053-I),0.02,0.02,0.02,1.00,47.37,0.02,inf
5,(WAG-F-LTE10-00-053-I),(WAG-F-LTE10-00-052-I),0.02,0.02,0.02,1.00,47.37,0.02,inf
6,"(WAG-F-LAG0-00-042-I, WAG-F-LTE10-00-052-I)",(WAG-F-LTE10-00-053-I),0.02,0.02,0.02,1.00,47.37,0.02,inf
7,"(WAG-F-LAG0-00-042-I, WAG-F-LTE10-00-053-I)",(WAG-F-LTE10-00-052-I),0.02,0.02,0.02,1.00,47.37,0.02,inf
8,"(WAG-F-LTE10-00-052-I, WAG-F-LTE10-00-053-I)",(WAG-F-LAG0-00-042-I),0.02,0.02,0.02,1.00,47.37,0.02,inf
9,(WAG-F-LAG0-00-042-I),"(WAG-F-LTE10-00-052-I, WAG-F-LTE10-00-053-I)",0.02,0.02,0.02,1.00,47.37,0.02,inf


# MT-SYSTEM的購物籃分析

In [189]:
client_dict = {}
for i, j in enumerate(df["Group"].unique().tolist()):
    print(i+1, j)
    client_dict.setdefault(i+1, j)
slct_client = int(input("請輸入要分析購物籃的客戶："))

1 Bourns
2 WE
3 Eaton
4 TELTONIKA
5 信昌
6 Laird
7 華新科技
8 MT-SYSTEM
9 LF
10 TE
11 藝感
12 LF OEM
13 M.S.
14 UnivaTe
15 SSF
16 G-PLAST
17 VTech
18 MATE
19 FIRSTEN
20 JBJ
21 Sancraft
22 iWave
23 Nexty
24 Zebra
請輸入要分析購物籃的客戶：1


In [190]:
client_filter = client_dict[slct_client]
sort_df = df[(df["預交日期"] >= "2020-01-01") & (df["Group"] == client_filter)]

In [191]:
order_numbers = sort_df["銷售單號"].unique().tolist()

In [192]:
cart_list = []
for i in order_numbers:
    order_list = sort_df[sort_df["銷售單號"] == i]["品名"].values
    cart_list.append(order_list)

In [193]:
te = TransactionEncoder()
te.ary = te.fit(cart_list).transform(cart_list)

In [194]:
te.columns_

['ACA-5036-A7-CC-S',
 'CI060303-3N9C*30kpcs(HOTAYI-TI',
 'EGA10402V05B0',
 'EGA10402V12B0',
 'EGA10402V24B0',
 'EGA10603V05B0',
 'EGA10603V12B0',
 'EGA10603V24B0',
 'EGA41206V12B0',
 'MCI0603HQ3N9CHBP',
 'MIP1608P1R0MBP',
 'MIP1608P2R2MBP',
 'MIP1608PR24MBP',
 'MIP1608PR47MBP',
 'MIP2012P1R0M05BP',
 'MIP2012P2R2M05BP',
 'MIP2012PR47M05BP',
 'MIP25201R0MBE',
 'MIP25201R5MBE',
 'MIP25202R2MBE',
 'MIP25203R3MBE',
 'MIP25204R7MBE',
 'MIP2520R47MBE',
 'MLVA0805M04-330',
 'MLVS0201V05330',
 'MLVS0402L04',
 'MLVS0402L14',
 'MLVS0402LE08',
 'MLVS0603AM14300',
 'MLVS0603AM20800',
 'MLVS0603AM25900',
 'MLVS0603L04',
 'MLVS0603L14',
 'MLVS0603L20',
 'MLVS0603LE08',
 'MLVS0603LE10',
 'MLVS1206AM30501',
 'MPA2016P2R2MBE',
 'MPA2016PR47MBE',
 'WFP201210Y-1R0ML',
 'WIP201610P-1R0ML',
 'WIP201610P-1R5ML',
 'WIP201610P-2R2ML',
 'WIP201610P-R33ML',
 'WIP201610P-R47ML',
 'WIP201610Y-1R0ML',
 'WIP201612S-1R0ML',
 'WIP201612S-R47ML',
 'WIP252012P-1R0ML',
 'WIP252012P-1R5ML',
 'WIP252012P-2R2ML',
 'WIP25201

In [195]:
test_frame = pd.DataFrame(te.ary, columns=te.columns_)

In [196]:
frequent_items = apriori(test_frame, min_support=0.18, use_colnames=True)
frequent_items["length"] = frequent_items["itemsets"].map(lambda x: len(x))

In [197]:
import plotly.express as px
px.histogram(frequent_items, x="support", nbins=10).show()


In [198]:
result = association_rules(frequent_items)
result.sort_values("confidence", ascending=False).reset_index(drop=True)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(EGA10603V12B0, MLVS0603L04)",(EGA10603V05B0),0.19,0.36,0.19,1.00,2.81,0.12,inf
1,(MLVS0603L04),(EGA10603V05B0),0.24,0.36,0.24,0.97,2.73,0.15,21.27
2,(MLVS0603LE08),(EGA10603V05B0),0.24,0.36,0.23,0.97,2.72,0.15,20.62
3,(MLVS0603L14),(EGA10603V05B0),0.24,0.36,0.23,0.94,2.64,0.14,10.63
4,(EGA10603V12B0),(EGA10603V05B0),0.28,0.36,0.26,0.92,2.59,0.16,8.16
5,(MIP25204R7MBE),(EGA10603V05B0),0.28,0.36,0.23,0.82,2.29,0.13,3.50
6,(MIP25202R2MBE),(EGA10603V05B0),0.27,0.36,0.22,0.81,2.28,0.12,3.41


In [199]:
import plotly.express as px
px.histogram(result, x="confidence", nbins=10).show()

In [200]:
test1 = pd.DataFrame(result, columns=result.columns)
test1["antecedents"] = test1["antecedents"].map(lambda x: list(x))
test1["consequents"] = test1["consequents"].map(lambda x: list(x))

In [201]:
x = sort_df.groupby(["品名"])[["數量", "本國幣別NTD"]].sum().reset_index()

In [202]:
total_volume = []
total_volume_consequents = []
total_revenue = []
total_revenue_consequents = []
for i in test1["antecedents"]:
    product_sum = 0
    for j in i:
#         print(j)
        volume = x[x["品名"] == j]["數量"].sum()
#         print(volume)
        product_sum += volume
    total_volume.append(product_sum)

for i in test1["antecedents"]:
    product_sum = 0
    for j in i:
#         print(j)
        volume = x[x["品名"] == j]["本國幣別NTD"].sum()
#         print(volume)
        product_sum += volume
    total_revenue.append(product_sum)

for i in test1["consequents"]:
    product_sum = 0
    for j in i:
#         print(j)
        volume = x[x["品名"] == j]["數量"].sum()
#         print(volume)
        product_sum += volume
    total_volume_consequents.append(product_sum)

for i in test1["consequents"]:
    product_sum = 0
    for j in i:
#         print(j)
        volume = x[x["品名"] == j]["本國幣別NTD"].sum()
#         print(volume)
        product_sum += volume
    total_revenue_consequents.append(product_sum)
    


In [203]:
test1["總數量"] = [i + j for i, j in zip(total_volume, total_volume_consequents)]

In [204]:
test1["總營業額"] = [k + y for k, y in zip(total_revenue, total_revenue_consequents)]

In [205]:
test1["antecedents_len"] = test1["antecedents"].map(lambda x: len(x))

In [106]:
ante_filter = int(input("請輸入先決條件數量："))
final_result = test1[test1["antecedents_len"] >= ante_filter]

請輸入先決條件數量：2


In [213]:
test1.style.format({"總數量": "{0:,.0f}", "總營業額": "{0:,.0f}"}).hide_index().background_gradient(subset=["總數量", "總營業額"], cmap="BuGn").set_caption("2020年Bourns購物籃分析")

antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,總數量,總營業額,antecedents_len
['EGA10603V12B0'],['EGA10603V05B0'],0.281481,0.355556,0.259259,0.921053,2.590461,0.159177,8.162963,"47,735,000","18,904,133",1
['MIP25202R2MBE'],['EGA10603V05B0'],0.274074,0.355556,0.222222,0.810811,2.280405,0.124774,3.406349,"41,417,000","15,657,354",1
['MIP25204R7MBE'],['EGA10603V05B0'],0.281481,0.355556,0.229630,0.815789,2.294408,0.129547,3.498413,"29,909,000","11,910,162",1
['MLVS0603L04'],['EGA10603V05B0'],0.244444,0.355556,0.237037,0.969697,2.727273,0.150123,21.266667,"51,538,000","13,738,039",1
['MLVS0603L14'],['EGA10603V05B0'],0.244444,0.355556,0.229630,0.939394,2.642045,0.142716,10.633333,"32,130,000","11,365,299",1
['MLVS0603LE08'],['EGA10603V05B0'],0.237037,0.355556,0.229630,0.968750,2.724609,0.145350,20.622222,"29,254,000","11,025,230",1
"['EGA10603V12B0', 'MLVS0603L04']",['EGA10603V05B0'],0.185185,0.355556,0.185185,1.000000,2.812500,0.119342,inf,"72,223,000","21,933,432",2


In [87]:
root = tk.Tk()
root.withdraw()
file_path = filedialog.askdirectory()
final_result.to_excel(file_path + "/teltonika_apriori.xlsx", index=False)